# Heiken Ashi Trading Strategy
This notebook implements and backtests a trading strategy based on **Heiken Ashi Indicators**.

### Strategy Logic:
- **Buy (Long Entry)**: On the first green Heiken Ashi candle (HA Close > HA Open).
- **Sell (Exit)**: On the first red Heiken Ashi candle (HA Close < HA Open) following a buy.

We use the project's built-in modular backtester with `strict_signals=True` to simulate discrete entry and exit events.


## 0) Setup
Imports from the local `src` directory.


In [ ]:
import os, sys
import numpy as np
import pandas as pd
from bokeh.io import output_notebook, show

# --- Setup correct working directory (ROOT) ---
if os.getcwd().endswith('notebooks'):
    os.chdir('..')

ROOT = os.getcwd()
if ROOT not in sys.path:
    sys.path.insert(0, ROOT)

from src.backtester.data import load_cleaned_assets, align_close_prices
from src.backtester.engine import BacktestConfig, run_backtest
from src.backtester.metrics import compute_performance_stats
from src.backtester.models import HeikenAshiMicroModel, combine_models_to_weights
from src.backtester.bokeh_plots import build_interactive_portfolio_layout
from src.backtester.report import compute_backtest_report

output_notebook()


## 0.1) Backtest Configuration
We enable `strict_signals=True` for event-driven entry/exit simulations.


In [ ]:
cfg = BacktestConfig(
    initial_equity=1_000_000,
    transaction_cost_bps=5,
    rebalance=None,  # Trade on signal changes
    mode='event_driven',
    strict_signals=True,
    stop_loss_pct=0.0
)
cfg


## 1) Data Loading


In [ ]:
assets = load_cleaned_assets(symbols=None)
close = align_close_prices(assets)
open_prices = pd.concat([df['Open'].astype(float).rename(s) for s, df in assets.items()], axis=1).sort_index()
high_prices = pd.concat([df['High'].astype(float).rename(s) for s, df in assets.items()], axis=1).sort_index()
low_prices = pd.concat([df['Low'].astype(float).rename(s) for s, df in assets.items()], axis=1).sort_index()
close.tail()


## 2) Market Proxy for Comparison
We compute a market proxy OHLCV for the Bokeh visualization.


In [ ]:
def build_market_proxy_ohlcv(assets, index):
    opens = pd.concat([df['Open'].astype(float).reindex(index) for df in assets.values()], axis=1).mean(axis=1)
    highs = pd.concat([df['High'].astype(float).reindex(index) for df in assets.values()], axis=1).mean(axis=1)
    lows = pd.concat([df['Low'].astype(float).reindex(index) for df in assets.values()], axis=1).mean(axis=1)
    closes = pd.concat([df['Close'].astype(float).reindex(index) for df in assets.values()], axis=1).mean(axis=1)
    return pd.DataFrame({'Open': opens, 'High': highs, 'Low': lows, 'Close': closes})

market_df = build_market_proxy_ohlcv(assets, close.index)
market_df.tail()


## 3) Heiken Ashi Strategy Implementation
We use the `HeikenAshiMicroModel` which calculates HA candles and discrete signals internally.


In [ ]:
model = HeikenAshiMicroModel()
# We scale signals by 0.1 to allocate 10% of portfolio per asset if multiple assets trigger
raw_signals = model.compute_signals(assets)
signals = raw_signals * 0.16 # Example allocation
signals.tail()


## 4) Backtest Execution


In [ ]:
print('Running Heiken Ashi Strategy...')
res = run_backtest(
    close_prices=close, 
    weights=signals, 
    config=cfg, 
    open_prices=open_prices,
    high_prices=high_prices,
    low_prices=low_prices
)
report = compute_backtest_report(result=res, close_prices=close, benchmark='equal_weight')
display(report)


## 5) Interactive Bokeh Visualization


In [ ]:
layout = build_interactive_portfolio_layout(
    market_ohlcv=market_df,
    equity=res.equity,
    returns=res.returns,
    weights=res.weights,
    turnover=res.turnover,
    costs=res.costs,
    close_prices=close, 
    title='Heiken Ashi Strategy'
)
show(layout)
